In [1]:
from ipywidgets import interact, widgets
interact(lambda x: x**2, x=widgets.IntSlider(min=0, max=10));

interactive(children=(IntSlider(value=0, description='x', max=10), Output()), _dom_classes=('widget-interact',…

# 📈 Scale Effects in the Romer Model

A key prediction of the basic Romer model of endogenous growth ($\dot{A}/A = \delta_A L_A$) is the presence of **scale effects**. This means that the long-run growth rate of the economy is predicted to be positively related to the *size* (or scale) of the economy, typically measured by the total population or labor force ($L$).

This notebook visualizes this relationship.

# ⚙️ Derivation and Intuition

Recall the key equations from the simple Romer model:
* Idea Growth Rate: $g_A = \dot{A}/A = \delta_A L_A$
* Labor Allocation: $L_A = \omega L$ (where $\omega$ is the constant fraction of labor in R&D)
* Output Per Capita Growth: $g_{Y/L} = g_A$

Substituting the labor allocation into the growth rate equation gives:
$$ g_A = g_{Y/L} = \delta_A \omega L $$

**The Scale Effect:** This equation directly implies that the long-run per capita growth rate ($g_{Y/L}$) is proportional to the total labor force ($L$).
* **Intuition:** More people ($L$) means more potential researchers ($L_A = \omega L$). More researchers lead to a faster rate of idea creation ($g_A$), which drives faster growth in output per person ($g_{Y/L}$).

**Prediction:** Larger countries (higher $L$) should have permanently higher per capita growth rates than smaller countries, assuming they have the same R&D productivity ($\delta_A$) and R&D share ($\omega$).

# 🌍 Empirical Evidence and Model Modifications

The strong scale effect predicted by this simple model is **not strongly supported by empirical evidence**. While larger markets might offer some advantages, we don't observe that large countries like China or India consistently grew much faster than smaller developed economies throughout history *solely* because of their size. Growth rates seem more independent of population size in the long run.

This has led to modifications of the Romer model, primarily by adjusting the idea production function:

* **"Fishing Out" / Dilution ($\phi < 1$):** $\dot{A} = \delta_A L_A A^{\phi}$. If $\phi < 1$, the exponent on $A$ makes it harder to find new ideas as the stock of knowledge grows. This can lead to a BGP where growth depends on population *growth* ($n$) rather than population *level* ($L$). This is characteristic of "semi-endogenous" growth models (like the Jones model).
* **Standing on Shoulders ($\phi > 1$):** If $\phi > 1$, existing ideas make future research *more* productive, potentially leading to accelerating growth.

The simulation below plots the relationship $g_Y = \delta_A \omega L$ predicted by the *simple* model, illustrating the scale effect concept.

In [2]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider, Layout
from IPython.display import display, Markdown
import warnings

# Optional: Use a specific style
try:
    plt.style.use('seaborn-v0_8-whitegrid')
except IOError:
    pass # Use default if style not found

def plot_scale_effects(delta_A=0.0005, omega=0.1, min_L=10.0, max_L=1000.0, num_points=30):
    """
    Plots the relationship between population size (L) and the predicted
    long-run growth rate (gY = delta_A * omega * L) in the simple Romer model.

    Args:
        delta_A (float): Productivity of R&D workers.
        omega (float): Fraction of labor force in R&D.
        min_L (float): Minimum population/labor force size to plot.
        max_L (float): Maximum population/labor force size to plot.
        num_points (int): Number of points (countries) to simulate along the range.
    """
    # Input validation
    num_points = int(num_points)
    if num_points < 2: num_points = 2
    min_L = max(min_L, 1)
    max_L = max(max_L, min_L + 1)
    delta_A = max(delta_A, 0)
    omega = np.clip(omega, 0.01, 0.99)

    # Generate a range of population sizes (L)
    L_values = np.linspace(min_L, max_L, num_points)

    # Calculate the predicted growth rate for each L
    # gY = gA = delta_A * omega * L
    gY_values = delta_A * omega * L_values

    # --- Plotting ---
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))

    # Plot 1: Growth Rate vs Population (Linear Scale)
    axes[0].plot(L_values, gY_values * 100, marker='o', linestyle='-', color='purple', markersize=6)
    axes[0].set_title("Scale Effect: Growth Rate vs Population (Linear)")
    axes[0].set_xlabel("Population / Labor Force (L)")
    axes[0].set_ylabel("Predicted Growth Rate (gY %)")
    axes[0].grid(True, linestyle='--', alpha=0.7)
    axes[0].yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y:.2f}%'))
    axes[0].set_xlim(left=0)
    axes[0].set_ylim(bottom=0)

    # Plot 2: Growth Rate vs Log Population
    log_L_values = np.log(L_values)
    axes[1].plot(log_L_values, gY_values * 100, marker='o', linestyle='-', color='teal', markersize=6)
    axes[1].set_title("Scale Effect: Growth Rate vs Log Population")
    axes[1].set_xlabel("Log Population / Labor Force (log L)")
    axes[1].set_ylabel("Predicted Growth Rate (gY %)")
    axes[1].grid(True, linestyle='--', alpha=0.7)
    axes[1].yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y:.2f}%'))
    axes[1].set_ylim(bottom=0)


    fig.suptitle(f"Romer Model Scale Effect: gY = δA * ω * L (δA={delta_A:.4f}, ω={omega:.2f})", fontsize=14, y=1.0)
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

    # --- Display Equation ---
    summary_md = f"""
    ### ⚙️ Scale Effect Equation:

    In this simple Romer model, the predicted long-run growth rate of output per capita ($g_{{Y/L}}$) is directly proportional to the size of the labor force ($L$):

    $$ g_{{Y/L}} = g_A = \\delta_A \\omega L $$

    * This implies larger countries should grow faster, all else equal.
    * The plot shows this linear relationship between $L$ and $g_Y$. Plotting against $\\log(L)$ shows the accelerating nature on a log scale for $L$.
    """
    display(Markdown(summary_md))


# --- Create Interactive Widgets ---
style = {'description_width': 'initial'}
layout = Layout(width='95%')

interact(
    plot_scale_effects,
    delta_A=FloatSlider(value=0.0005, min=0.0, max=0.005, step=0.0001, description='R&D Productivity (δA):', style=style, layout=layout, readout_format='.4f'),
    omega=FloatSlider(value=0.1, min=0.01, max=0.5, step=0.01, description='R&D Share (omega ω):', style=style, layout=layout, readout_format='.2f'),
    min_L=FloatSlider(value=10.0, min=1, max=100, step=1, description='Min Population (L):', style=style, layout=layout, readout_format='.0f'),
    max_L=FloatSlider(value=1000.0, min=100, max=5000, step=50, description='Max Population (L):', style=style, layout=layout, readout_format='.0f'),
    num_points=IntSlider(value=30, min=5, max=100, step=5, description='# Points (Countries):', style=style, layout=layout)
);


interactive(children=(FloatSlider(value=0.0005, description='R&D Productivity (δA):', layout=Layout(width='95%…

# 📘 Scale Effects Hypothesis (Romer, GrowthEcon Ch. 7.4)

From the Romer model:
\[
g_Y = z \cdot \omega \cdot L
\]

- More people → more researchers → faster growth
- This predicts a **positive relationship** between \( \log(L) \) and \( g_Y \)

🔑 **Key Insight**:  
Scale effects mean large countries should grow faster — but this is debated in modern growth theory.

**Sources**:  
- GrowthEcon [Ch. 7.4](https://growthecon.com/StudyGuide/romer-scale.html)  
- Charles Jones, *Macroeconomics*, Ch. 8